In [1]:
# from pymongo import MongoClient
# client = MongoClient('mongodb://127.0.0.1', 27017)
# # Access/Initiate Database
# db = client.ebay
# # Access/Initiate Table
# tab = db.table


In [1]:
import warnings
warnings.filterwarnings('ignore')
import copy
import pandas as pd
import numpy as np
# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import pprint

In [2]:
doordash_url='doordash-menu'
# grubhub_url = 'https://www.grubhub.com/restaurant/south-bay-rice-things-2401-artesia-blvd-redondo-beach/1728314?gclid=EAIaIQobChMI0vu-7qin6AIVZv_jBx0RsQZVEAAYASAAEgLMsvD_BwE&gclsrc=aw.ds'
with open(doordash_url, 'r') as f:
    s= f.read()
soup = BeautifulSoup(s, "html")


In [3]:
menu = dict()
for cat in soup.find_all('div',{'class':'sc-dwztqd bXZThm'}):
    category = list(cat.find('h2').children)[0]
    print('\n\n---------------------')
    print(category)
    print('---------------------')
    for div in cat.find_all("div", {'data-anchor-id':"MenuItem"}):
    #     button = menu_item.find('button',{'type':'button'})#.prettify()

        item = div.find('button',{'type':'button'}).prettify().split('"')[1].split('$')
        name, price = item[0][:-1],float(item[1])

        print(name,price)    

        try:

            desc = div.find('span',{'display':'block'}) #.prettify()
            description = desc.find('span',{'color':"#767676"}).prettify().split('>')[1].split('<')[0].strip().replace('\n','')
        except:
            description = ''        
        try:
            pic_div = div.find('picture').prettify()
    #         print(pic_div)
            pic = pic_div.split('"')[5].split(',')[5].split('auto')[1].split(' ')[0][1:]
            menu[name] = [category,price,description,pic]
        except:
            menu[name] = [category,price,description,'']   

#     print(desc)


# print(menu)



---------------------
Popular Items
---------------------
California 5.18


---------------------
Salads
---------------------
Calamari Salad 8.63
Salmon Skin Salad 8.63


---------------------
Sushi Roll
---------------------
California 5.18
Spicy California 5.18
Vegetable Roll 5.18
Cucumber Roll 5.18
Avocado Roll 5.18
Inari (3pc) 5.18
Tuna Roll 6.9
Spicy Tuna Roll 6.9
Salmon Roll 6.9
Salmon Avocado 6.9
Philly Roll 6.9
Crunchy (5pc) 6.9
Veggie Crunchy Roll (5pc) 6.9
Eel Roll 8.05
Spicy Eel Roll 8.05
Eel Avocado 8.05


---------------------
Premium Sushi
---------------------
Salmon Crunchy 10.35
Salmon Skin Roll (5pc) 10.35
California Tempura 10.35
Super California 10.35
Eel Tempura 10.35
Alaskan 10.35
Hawaiian 10.35
Caterpillar 10.35


---------------------
Fresh Sushi Bowl
---------------------
Tuna Bowl 12.65
Spicy Tuna Bowl 12.65
Salmon Bowl 12.65
Chirashi Bowl 18.4


---------------------
Combo Plate
---------------------
Combo Plate (1 Item) 6.9
Combo Plate (2 Item Combo) 9.78

In [114]:
def alter_columns(x):
    if x['name'] == 'Combo Plate (2 Item Combo)':
        x['name'] = 'Combo Plate (2 Items)'
    if x['name'] == 'Crunchy (5pc)':
        x['name'] = 'Crunchy Roll (5pc)'
    elif x['name'] == 'Inari (3pc)':
        x['name'] = 'Inari Roll(3pc)'
    x['name'] = x['name'].replace('Teri','Teriyaki')
    if x['image'] == '':
        x['image'] = 'images/'+x['category'].replace(' ','')+'.jpg'
    if x['name'].find('(Dark Meat)')!=-1:
        x['name'] = x['name'].replace(' (Dark Meat)','')
        x['description'] = ('Dark Meat ' + x['name']).capitalize()
    if x['name'].find('(White Meat)')!=-1:
        x['name'] = x['name'].replace(' (White Meat)','')
        x['description'] = ('White Meat ' + x['name']).capitalize()
    if x['description'] == '3 Item Combo':
        x['description'] = '3 Item Combo Plate'
    if x['description'].find('Combo')!=-1:
        x['description'] = x['description'].replace('Combo','Combination').capitalize()
    return x

In [115]:
# for m in menu.items():
#     print(m)

df = pd.DataFrame.from_dict(menu,orient='index')
df.reset_index(inplace=True)
df.columns = ['name','category','price','description','image']
## Make Edits
df.loc[df.category=='Udon Noodle','name'] = [name + ' Udon' for name in df[df.category=='Udon Noodle']['name']]
df.loc[df.category=='Ramen Noodle','name'] = [name + ' Ramen' for name in df[df.category=='Ramen Noodle']['name']]
df.loc[df.category=='Sushi Roll','name'] = [name + ' Roll' if name.find('Roll')==-1 and name.find('pc)')==-1 else name for name in df[df.category=='Sushi Roll']['name'] ]
df.loc[df.category=='Premium Sushi','name'] = [name + ' Roll' if name.find('Roll')==-1 and name.find('pc)')==-1 else name for name in df[df.category=='Premium Sushi']['name'] ]
df = df.apply(lambda x: alter_columns(x),axis=1)
df[:60]

,name,category,price,description,image
0,California Roll,Sushi Roll,5.18,"Crab Mix, Mayo, Cucumber, Avocado",https://cdn.doordash.com/media/photos/e820e6e1...
1,Calamari Salad,Salads,8.63,,images/Salads.jpg
2,Salmon Skin Salad,Salads,8.63,,images/Salads.jpg
3,Spicy California Roll,Sushi Roll,5.18,"Crab Mix, Mayo, Cucumber, Avocado, Spicy Sauce...",images/SushiRoll.jpg
4,Vegetable Roll,Sushi Roll,5.18,"Cucumber, Avocado, Kaiware Sprouts, Pickled Ya...",images/SushiRoll.jpg
5,Cucumber Roll,Sushi Roll,5.18,Cucumber,images/SushiRoll.jpg
6,Avocado Roll,Sushi Roll,5.18,Avocado,images/SushiRoll.jpg
7,Inari Roll(3pc),Sushi Roll,5.18,"Fried Tofu Skin, Sweet Soy Flavored, Sushi Rice",images/SushiRoll.jpg
8,Tuna Roll,Sushi Roll,6.90,"Freshwater Tuna, Cucumber",images/SushiRoll.jpg
9,Spicy Tuna Roll,Sushi Roll,6.90,"Freshwater Tuna, Cucumber, Spicy Sauce Blend",images/SushiRoll.jpg


In [116]:
text = '''
    <div class="col-md-4 text-center">
        <div class="menu-wrap">
            <a href="#" class="menu-img img mb-4" style="background-image: url(images/IMAGEPATH.jpg);"></a>
            <div class="text">
                <h3><a href="#">FOODITEM</a></h3>
                <p>DESCRIPTION</p>
                <p class="price"><span>$PRICE</span></p>
            </div>
        </div>
    </div>'''


In [117]:
def make_html(x):
    price = '{value:.2f}'.format(value=x['price'])
    item_text = text.replace('FOODITEM',x['name'])
    item_text = item_text.replace('IMAGEPATH',x['image'])
    item_text = item_text.replace('DESCRIPTION',x['description'])
    item_text = item_text.replace('PRICE',price)
    print(item_text)
#     return item_text

In [127]:
categories = df.category.unique()
imgnames = [cat.replace(' ','')+'.jpg' for cat in categories]
[print(x) for x in list(zip(categories,imgnames))]
print()

('Sushi Roll', 'SushiRoll.jpg')
('Salads', 'Salads.jpg')
('Premium Sushi', 'PremiumSushi.jpg')
('Fresh Sushi Bowl', 'FreshSushiBowl.jpg')
('Combo Plate', 'ComboPlate.jpg')
('Ramen Noodle', 'RamenNoodle.jpg')
('Udon Noodle', 'UdonNoodle.jpg')
('Poke Bowl', 'PokeBowl.jpg')
('Curry Plate', 'CurryPlate.jpg')
('Side Orders', 'SideOrders.jpg')
('Beverages', 'Beverages.jpg')



['SushiRoll.jpg',
 'Salads.jpg',
 'PremiumSushi.jpg',
 'FreshSushiBowl.jpg',
 'ComboPlate.jpg',
 'RamenNoodle.jpg',
 'UdonNoodle.jpg',
 'PokeBowl.jpg',
 'CurryPlate.jpg',
 'SideOrders.jpg',
 'Beverages.jpg']

In [55]:
for i,cat in enumerate(categories):
    print('<!--',cat,' -->')
    mytext = '''

<div class="tab-pane fade show active" id="v-pills-NUM" role="tabpanel" aria-labelledby="v-pills-NUM-tab">
  <div class="row">    
    '''
    print(mytext.replace('NUM',str(i)))
    df[df.category==cat].apply(make_html,axis=1)
    mytext = '''
  </div>
</div>    
'''
    print(mytext)

<!-- Sushi Roll  -->


<div class="tab-pane fade show active" id="v-pills-0" role="tabpanel" aria-labelledby="v-pills-0-tab">
  <div class="row">    
    

    <div class="col-md-4 text-center">
        <div class="menu-wrap">
            <a href="#" class="menu-img img mb-4" style="background-image: url(images/SushiRoll.jpg);"></a>
            <div class="text">
                <h3><a href="#">California</a></h3>
                <p>Crab Mix, Mayo, Cucumber, Avocado</p>
                <p class="price"><span>$5.18</span></p>
            </div>
        </div>
    </div>

    <div class="col-md-4 text-center">
        <div class="menu-wrap">
            <a href="#" class="menu-img img mb-4" style="background-image: url(images/SushiRoll.jpg);"></a>
            <div class="text">
                <h3><a href="#">Spicy California</a></h3>
                <p>Crab Mix, Mayo, Cucumber, Avocado, Spicy Sauce Blend</p>
                <p class="price"><span>$5.18</span></p>
            </div>
    

### Alternate menu

In [59]:
text_alternate = '''
    <div class="pricing-entry d-flex ftco-animate">
        <div class="img" style="background-image: url(images/IMAGEPATH.jpg);"></div>
        <div class="desc pl-3">
            <div class="d-flex text align-items-center">
                <h3><span>FOODITEM</span></h3>
                <span class="price">$PRICE</span>
            </div>
            <div class="d-block">
                <p>DESCRIPTION</p>
            </div>
        </div>
    </div>
'''

In [61]:
def make_html_alternate(x):
    price = '{value:.2f}'.format(value=x['price'])
    item_text = text_alternate.replace('FOODITEM',x['name'])
    item_text = item_text.replace('IMAGEPATH',x['category'].replace(' ',''))
    item_text = item_text.replace('DESCRIPTION',x['description'])
    item_text = item_text.replace('PRICE',price)
    print(item_text)
#     return item_text

In [63]:
for i,cat in enumerate(categories):
    print('<!--',cat,' -->')
    mytext = '''
<div class="col-md-7 heading-section text-center ftco-animate">
	<h2 class="mb-4">CATEGORY</h2>
</div>
<div class="row">
    <div class="col-md-6">
    '''
    print(mytext.replace('CATEGORY',cat))
    df[df.category==cat].apply(make_html_alternate,axis=1)
    mytext = '''
  </div>
</div>    
'''
    print(mytext)

<!-- Sushi Roll  -->


<div class="row">
    <div class="col-md-6">
    

    <div class="pricing-entry d-flex ftco-animate">
        <div class="img" style="background-image: url(images/SushiRoll.jpg);"></div>
        <div class="desc pl-3">
            <div class="d-flex text align-items-center">
                <h3><span>California</span></h3>
                <span class="price">$5.18</span>
            </div>
            <div class="d-block">
                <p>Crab Mix, Mayo, Cucumber, Avocado</p>
            </div>
        </div>
    </div>


    <div class="pricing-entry d-flex ftco-animate">
        <div class="img" style="background-image: url(images/SushiRoll.jpg);"></div>
        <div class="desc pl-3">
            <div class="d-flex text align-items-center">
                <h3><span>Spicy California</span></h3>
                <span class="price">$5.18</span>
            </div>
            <div class="d-block">
                <p>Crab Mix, Mayo, Cucumber, Avocado, Spicy Sa